In [ ]:
import subprocess
from tokenize import String
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from collections import namedtuple
import os

DAWNWARD = "/home/luca/Desktop/Projects/Automated-Planning/Dawnward/downward/fast-downward.py"


# Planner


## Plan request

In [ ]:
# Available planner with dawnward (locally)
# lama
# lama-first
#seq-opt-bjolp
#seq-opt-fdss-1
#seq-opt-fdss-2
#seq-opt-lmcut
#seq-opt-merge-and-shrink
#seq-sat-fd-autotune-1
#seq-sat-fd-autotune-2
#seq-sat-fdss-1
#seq-sat-fdss-2
#seq-sat-fdss-2014
#seq-sat-fdss-2018
#seq-sat-fdss-2023
#seq-sat-lama-2011

def request_plan(alias, evaluators, search, problem, domain):
    # Define the command to execute Fast Downward
    command = [DAWNWARD]

    if (alias != None):
        command.append('--alias')
        command.append(alias)

    command.append(domain)
    command.append(problem)
    
    if (evaluators != None):
        for eval in evaluators:
            command.append('--evaluator')
            command.append(eval)
            
    if (search != None):
        command.append('--search')
        command.append(search)
        
    print(f'Command performed:\n', command, '\n', '.' + ' '.join(command))
    # Execute the command and capture the output
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    output_lines = process.stdout.splitlines()
    print(process.stdout)
    plan_start = False
    plan = []
    for line in output_lines:
        if "Solution found!" in line:
            plan_start = True
        elif plan_start:
            if line.startswith(";"):
                break  # Fine del piano
            #print(line)
            plan.append(line)
            
    return plan[1:]

# Problem 1

## Environment generation

In [ ]:
Position = namedtuple('Position', ['x','y'])
EnvironmentConfig = namedtuple('EnvironmentConfig', ['X', 'Y', 'active_cells', 'agents',
                                                     'content', 'boxes', 'workstations', 'warehouses'])

def select_random_one_position(matrix, exclude_positions=[]):
    # Trova le posizioni dei valori impostati ad 1
    ones_positions = np.where(np.logical_or(matrix == 1, matrix == 3))
    ones_list = [Position(x,y) for (x,y) in zip(ones_positions[0], ones_positions[1])] # Crea una lista di tuple (x, y)

    # Filtra la lista di posizioni escludendo quelle nella lista di esclusione
    filtered_ones_list = [pos for pos in ones_list if pos not in exclude_positions]

    if not filtered_ones_list:
        return None  # Restituisce None se non ci sono posizioni valide dopo l'esclusione

    # Seleziona una posizione casuale tra quelle rimanenti
    return random.choice(filtered_ones_list)


def print_colored_matrix_seaborn(matrix, title):
    # Definizione della palette di colori: uno per ogni valore unico nella matrice
    # 0: bianco, 1: verde, 2: blu, 3: rosso
    colors = ["white", "green", "blue", "red"]
    # Crea un colormap personalizzato da una lista di colori
    cmap = sns.color_palette(colors)
    plt.figure(figsize=(20, 15))
    plt.title(title)
    
    annot_matrix = matrix
    color_matrix = np.where(matrix != 0, np.where(matrix % 3 == 0, 3, matrix), matrix)

    # Crea una mappa di calore con Seaborn
    sns.heatmap(color_matrix, annot=annot_matrix, cmap=cmap, cbar=False, linewidths=.5,
                linecolor='gray', square=True, fmt="d")
    plt.show()
    
class Agent:
    
    def __init__(self, id, position):
        self.id = id
        self.position = position

    def __str__(self):
        return f'[{self.id}, {self.position}]'
    __repr__ = __str__

class Warehouse:
    
    def __init__(self, id, position):
        self.id = id
        self.position = position
        
    def __str__(self):
        return f'[{self.id}, {self.position}]'
    __repr__ = __str__

class Workstation:
    
    def __init__(self, id, position, types):
        self.id = id
        self.position = position
        self.own_supplies = { type : [] for type in types}
        self.needed_supplies = { type : 0 for type in types}
        
    def __str__(self):
        return f'[{self.id}, {self.position}, O:{self.own_supplies}, N:{self.needed_supplies}]'
    __repr__ = __str__

    
class Supply:

    def __init__(self, id, position, type, stored_by):
        self.id = id
        self.position = position 
        self.type = type
        self.stored_by = stored_by
        
    def __str__(self):
        return f'[{self.id}, {self.position}, {self.type}]'
    __repr__ = __str__
    
    def update_location(self, position, stored_by):
        self.position = position
        self.stored_by = stored_by
    
class Box:
    
    def __init__(self, id, position, initial_wh_id):
        self.id = id
        self.position = position
        self.warehouse_id = initial_wh_id

    def __str__(self):
        return f'[{self.id}, {self.position}]'
    __repr__ = __str__

class Environment:
    
    def __init__(self, configuration, init_type = 'P1', verbose = True):
            
        self.configuration = configuration
        self.X = configuration.X
        self.Y = configuration.Y
        self.active_cells = configuration.active_cells
        self.agent_count = configuration.agents
        self.warehouses_count = configuration.warehouses
        self.workstation_count = configuration.workstations
        self.supplies = configuration.content
        self.box_count = configuration.boxes
        self.matrix = np.zeros((self.X, self.Y), dtype=int)
        
        self.supply_types = [type for type, qty in self.supplies]
        self.supply_type_count = len(self.supplies)
        self.supplt_qty_tot = np.sum([qty for _, qty in self.supplies])
        self.warehouse_positions = []
        
        self.agents = {}
        self.warehouses = {}
        self.workstations = {}
        self.objects = {}
        for type in self.supply_types: self.objects[type] = {}
        self.boxes = {}
        
        self.create_environment_active_area()

        self.generate_warehouses()
        self.generate_agents()
        self.generate_content()
        self.generate_boxex()
        self.generate_workstations()
        
        self.random_goal()
        self.workstations_state = {id : None for id in self.workstations.keys()}
        
        for ws in self.workstations.values():
            ws_own_supplies = { type : len(objs) for type, objs in ws.own_supplies.items()}
            ws_needs = { type : own_qty + need_qty for (type, own_qty), (_, need_qty) in zip(ws_own_supplies.items(), ws.needed_supplies.items())}

            self.workstations_state[ws.id] = {'own': ws_own_supplies, 'needs': ws_needs}
                
        if verbose: self.toString()

    def random_goal(self):
        # Mantieni un elenco di oggetti non ancora assegnati per ogni tipo
        cloned_objects = { type : [obj for obj in self.objects[type].values()] for type in self.supply_types}
        
        def update_content(cloned_objects, i):
            available_content = {supply_type : {
                'qty' : len(cloned_objects[supply_type]),
                'max_qty': int(len(cloned_objects[supply_type]) / (len(self.workstations) - i))} 
                for supply_type in self.supply_types if len(cloned_objects[supply_type]) > 0}  
            
            available_types = len(available_content.keys())
            types = list(available_content.keys())
            return available_content, available_types, types
        
        for i, ws in enumerate(self.workstations.values()):
            
            available_content, available_types, types = update_content(cloned_objects, i)

            # Decisione casuale sul se la workstation possiede già delle forniture
            if random.randint(0, 100) > 40:  # 60% di probabilità
                supply_types_own = random.sample(types, k = random.randint(1, available_types))
                for supply_type in supply_types_own:
                    supply_quantity_own = random.randint(1, available_content[supply_type]['max_qty'])
                    available_content[supply_type]['qty'] -= supply_quantity_own
                    for _ in range(supply_quantity_own):
                        obj = cloned_objects[supply_type].pop()  # Rimuovi l'oggetto dall'elenco degli non assegnati
                        self.objects[supply_type][obj.id].update_location(ws.position, ws.id)
                        ws.own_supplies[supply_type].append(obj)

            available_content, available_types, types = update_content(cloned_objects, i)

            if random.randint(0, 100) > 10:  # 90% di probabilità
                supply_types_needed = random.sample(types, k = random.randint(1, available_types))
                for supply_type in supply_types_needed:
                    #print(available_content)
                    supply_quantity_needed = random.randint(1, available_content[supply_type]['max_qty'])
                    available_content[supply_type]['qty'] -= supply_quantity_needed
                    ws.needed_supplies[supply_type] = supply_quantity_needed
                               
    def create_environment_active_area(self):
            # Sceglie un punto di partenza casuale e imposta la cella a 1
            current_position = Position(random.randint(0, self.X - 1), random.randint(0, self.Y - 1))
            self.matrix[current_position.x, current_position.y] = 1
            steps_taken = 1

            while steps_taken < self.active_cells:
                # Definisce i possibili movimenti: su, giù, sinistra, destra
                movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]
                random.shuffle(movements)

                for dx, dy in movements:
                    next_position = Position(current_position.x + dx, current_position.y + dy)

                    if next_position != current_position and 0 <= next_position[0] < self.X and 0 <= next_position[1] < self.Y and self.matrix[next_position] == 0:
                        self.matrix[next_position] = 1
                        current_position = next_position
                        steps_taken += 1
                        break
                    else:
                        # Se tutte le direzioni sono bloccate, trova un nuovo punto di partenza tra quelli già impostati a 1
                        ones = np.argwhere(self.matrix == 1)
                        random_index = random.randint(0, len(ones) - 1)
                        current_position = Position(ones[random_index][0], ones[random_index][1])
            
    def generate_agents(self):
        for i in range(self.agent_count):
            
            wh = random.choice(list(self.warehouses.values())) # Initialize agent at a random warehouse - ?
            agent = Agent(f'agent_{i}', wh.position)
            self.agents[f'agent_{i}'] = agent
            
    def generate_warehouses(self):
        for i in range(self.warehouses_count): 
            
            pos = select_random_one_position(self.matrix, self.warehouse_positions)
            wh = Warehouse(f'warehouse_{i}', pos)
            self.warehouses[f'warehouse_{i}'] = wh
            
            self.warehouse_positions.append(pos)
            self.matrix[pos] = 2

    def generate_workstations(self):
        
        for i in range(self.workstation_count):
            
            pos = select_random_one_position(self.matrix, self.warehouse_positions)
            ws = Workstation(f'workstation_{i}', pos, self.supply_types)
            
            self.workstations[f'workstation_{i}'] = ws

            # Matrix set
            if (self.matrix[pos] % 3 == 0): self.matrix[pos] += 3
            else: self.matrix[pos] = 3

    def generate_content(self):
        
        for (type, qty) in self.supplies:
            
            for q in range(qty):
                wh = random.choice(list(self.warehouses.values()))
                supply = Supply(f'obj_{type}_{q}', wh.position, type, wh.id)
                self.objects[type][f'obj_{type}_{q}'] = supply

    def generate_boxex(self):

        for i in range(self.box_count):
            wh = random.choice(list(self.warehouses.values()))
            box = Box(f'box_{i}', wh.position, wh.id)
            self.boxes[f'box_{i}'] = box
           
    def toString(self):
        print('Environment info:')
        print(f' - Environment size: {self.X}x{self.Y}\n - Active area: {self.active_cells} ({(self.active_cells / (self.X * self.Y))*100}%)')
        print(f' - Warehouses: {self.warehouses_count}')
        print(f' - Agents: {self.agent_count}')
        print(f' - Workstations: {self.workstation_count}')
        print(f' - Content types: {self.supply_type_count}')
        print(f' - Content total quantity: {self.supplies}')
        print(f' - Boxes available: {len(self.boxes)}')
        print('Examples:')
        print(f' - Warehouse encoded:', list(self.warehouses.values())[0])
        print(f' - Agent encoded:', list(self.agents.values())[0])
        print(f' - Workstation encoded:', list(self.workstations.values())[0])
        print(f' - Content encoded:', list(self.objects.items())[0])
        print(f' - Box encoded:', list(self.boxes.values())[0])
        print('Workstation goal & state:')
        for ws_id, state in self.workstations_state.items():
            print(f' - {ws_id}, own',state['own'],'supplies - Needs ',state['needs'])
        print_colored_matrix_seaborn(self.matrix, 'Environment')
    
class Environment_PDDL:
    
    def __init__(self, env : Environment, path):
        self.baseline_path = path
        self.id = str(self.generate_id())
        self.environment = env
        self.init = set()
        self.goals = set()
        self.pddl_objects = {
            'location' : [],
            'agent' : [],
            'supply' : { type : [] for type in env.supply_types},
            'box' : [],
            'workstation' : [],
            'warehouse' : []
        }

        self.pddl_facts = {
            'location' : set(),
            'agent' : set(),
            'supply' : { type : set() for type in env.supply_types},
            'box' : set(),
            'workstation' : set(),
            'warehouse' : set()
        }
        
        for x in range(env.matrix.shape[0]):  # matrice.shape[0] restituisce il numero di righe
            for y in range(env.matrix.shape[1]):
                 if (env.matrix[x][y] != 0):
                    
                    self.pddl_objects['location'].append(f'l{x}_{y}')

                    adjacent_positions = [Position(x - 1, y), Position(x + 1, y), Position(x, y - 1), Position(x, y + 1)]
                    for p in adjacent_positions:

                        if 0 <= p.x < env.X and 0 <= p.y < env.Y and env.matrix[p] != 0:
                            self.pddl_facts['location'].add(f'adjacent l{x}_{y} l{p.x}_{p.y}')
                            self.pddl_facts['location'].add(f'adjacent l{p.x}_{p.y} l{x}_{y}')
                            self.init.add(f'adjacent l{x}_{y} l{p.x}_{p.y}')
                            self.init.add(f'adjacent l{p.x}_{p.y} l{x}_{y}')
                            
        for agent in env.agents.values():
            self.pddl_objects['agent'].append(agent.id)
            self.pddl_facts['agent'].add(f'at {agent.id} l{agent.position.x}_{agent.position.y}')
            self.init.add(f'at {agent.id} l{agent.position.x}_{agent.position.y}')
            
        for wh in env.warehouses.values():
            self.pddl_objects['warehouse'].append(wh.id)
            self.pddl_facts['warehouse'].add(f'at {wh.id} l{wh.position.x}_{wh.position.y}')
            self.init.add(f'at {wh.id} l{wh.position.x}_{wh.position.y}')
            
        for ws in env.workstations.values():
            self.pddl_objects['workstation'].append(ws.id)
            self.pddl_facts['workstation'].add(f'at {ws.id} l{ws.position.x}_{ws.position.y}')
            self.init.add(f'at {ws.id} l{ws.position.x}_{ws.position.y}')
            
            state = env.workstations_state[ws.id]
            for type, count in state['own'].items():
                self.pddl_facts['workstation'].add(f'= (num_objects_at {ws.id} {type}) {count}')
                self.init.add(f'= (num_objects_at {ws.id} {type}) {count}')
                
            for type, count in state['needs'].items():   
                self.goals.add(f'= (num_objects_at {ws.id} {type}) {count}')
                               
        for box in env.boxes.values():
            self.pddl_objects['box'].append(box.id)
            self.pddl_facts['box'].add(f'at {box.id} l{box.position.x}_{box.position.y}')
            self.pddl_facts['box'].add(f'empty {box.id}')
            self.pddl_facts['box'].add(f'contains {box.warehouse_id} {box.id}')
            self.init.add(f'at {box.id} l{box.position.x}_{box.position.y}')
            self.init.add(f'empty {box.id}')
            #self.init.add(f'has {box.warehouse_id} {box.id}')
            
        for type, objects in env.objects.items():
            for object in objects.values():
                self.pddl_objects['supply'][type].append(object.id)
                self.pddl_facts['supply'][object.type].add(f'at {object.id} l{object.position.x}_{object.position.y}')
                self.pddl_facts['supply'][object.type].add(f'has {object.stored_by} {object.id}')
                self.init.add(f'at {object.id} l{object.position.x}_{object.position.y}')
                #self.init.add(f'has {object.stored_by} {object.id}')
            
        print('PDDL:')
        print(f' - PDDL objects:')
        for (key, objs) in self.pddl_objects.items():
            print(f' -- {key}:', objs)
            
        print(f' - PDDL facts:')
        for (key, facts) in self.pddl_facts.items():
            print(f' -- {key}:', facts)
            
        print(' - PDDL goals:')
        for fact in self.goals:
            print(f' - {fact}')
            
        self.generate_problem()
        
    def generate_problem(self):
        self.problem_init = ' '.join([ f'({fact})' for fact in self.init])
        self.problem_objects = ''
        for type, objects in self.pddl_objects.items():
            
            if type == 'supply':
                for s_type, s_objects in objects.items():
                    self.problem_objects += ' '.join(s_objects) + f' - {s_type} '
            else:
                self.problem_objects += ' '.join(objects) + f' - {type} '
               
        self.problem_goal = 'and ' + ' '.join([f'({goal})' for goal in self.goals])
        self.pddl_problem = (';; problem file: problem_' + self.id + '.pddl\n' +
                '(define (problem default)\n' +
                '  (:domain default)\n' +
                '  (:objects ' + self.problem_objects + ')\n' +
                '  (:init ' + self.problem_init + ')\n' +
                '  (:goal (' + self.problem_goal + '))\n)')
    
        self.save_problem()
        
    def generate_id(self):
        max_index = -1  # Inizia da -1 per gestire il caso in cui non ci siano cartelle
        for item in os.listdir(self.baseline_path):
            full_path = os.path.join(self.baseline_path, item)
            # Controlla solo le cartelle, ignora i file
            if os.path.isdir(full_path) and item.startswith("environment_"):
                index = int(item.split('_')[1])
                max_index = max(max_index, index)
        # L'indice del prossimo ambiente sarà il massimo indice trovato + 1
        return str(max_index + 1)  # Restituisce l'ID come stringa

        
    def save_problem(self):
        # Crea il percorso della cartella in cui salvare il problema
        env_path = os.path.join(self.baseline_path, f"environment_{self.id}")
        # Crea la cartella se non esiste
        os.makedirs(env_path, exist_ok=True)

        # Percorso completo del file da salvare
        file_path = os.path.join(env_path, "problem.pddl")

        # Salva il problema PDDL nel file
        with open(file_path, 'w') as file:
            file.write(self.pddl_problem)
        
        print(f"Il problema PDDL è stato salvato come: {file_path}")         

## Main

In [ ]:

# Define the paths to your domain and problem PDDL files
PROBLEM = 'problem_1'
DOMAIN_FILE_PATH = f'{PROBLEM}/domain.pddl'

#PROBLEM_FILE_PATH = None #"/home/luca/Desktop/Projects/Automated-Planning/Project/pddl-files/examples-0/fridge/problem.pddl"
# Define the path to the Fast Downward script


environment_config = EnvironmentConfig(X=5, 
                                       Y=5, 
                                       active_cells=10, 
                                       agents = 1,
                                       boxes=2, 
                                       workstations=1, 
                                       warehouses=1,
                                    content = [('valve',2), ('bolt',2), ('tool',2)])

test_env = Environment(environment_config, verbose=True)
test_env_pddl = Environment_PDDL(test_env, 'problem_1/environments')


In [ ]:
SEARCH = 'lazy_greedy([hff, hcea], preferred=[hff, hcea])'
ALIAS = None #"seq-sat-lama-2011"
EVALUATORs =  ['hff=ff()','hcea=cea()']

plan = request_plan(ALIAS, EVALUATORs, SEARCH, test_env_pddl.pddl_problem, DOMAIN_FILE_PATH)
print(f'Steps: {len(plan)}')
print(plan)
for i, step in enumerate(plan):
   print(f'Step {i} - Action: ', step.split(' ')[0],' - Args: ', step.split(' ')[1])